## 모델 훈련

모델을 만들고, 어떻게 학습하는지 원리에 대해 이해했다면, 이제 모델을 학습시켜봅시다.

데이터부터 준비하겠습니다

In [150]:
from zipfile import ZipFile
import pandas as pd
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as f:
    data=io.BytesIO(f.read("고객 대출등급 분류 해커톤/train.csv"))

data=pd.read_csv(data)

In [151]:
import keras 
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [180]:
from sklearn import model_selection as mod
import numpy as np

num_cols=list(data.select_dtypes(np.number).columns)
cate_cols=["대출기간","근로기간","주택소유상태","대출목적"]

use_feature=data[num_cols+cate_cols]
label_list=data.대출등급.unique()
label_list.sort()

use_label=data.대출등급.map(lambda label : np.argmax(label_list==label))

train_feature,test_feature,train_label,test_label=mod.train_test_split(
    use_feature,use_label,train_size=0.9,stratify=use_label,random_state=1)

조금 복잡한 모델을 만들어보죠
(Wide&Deep Model)

이 모델은 input을 두가지 경로로 받습니다.

In [181]:
norm_layer=keras.layers.Normalization()
norm_layer.adapt(data[num_cols].values)

cate_layer=keras.layers.StringLookup()
cate_layer.adapt(data[cate_cols].values)

In [154]:
cate_layer(data[cate_cols].values[:3])

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 1, 19,  4,  2],
       [ 5,  6,  3, 13],
       [ 1, 15,  3,  2]])>

위의 층간 특징은 따로 다루겠습니다.

In [155]:
class WD_Model(keras.Model):
    def __init__(self,layer_nums:list ,*args, **kwargs):
        super().__init__(*args, **kwargs)

        self.deep_hidden_layer=keras.layers.Dense(9,activation="relu")
        self.wide_hidden_layer=keras.layers.Dense(4,activation="relu")

        self.deep_num,self.wide_num=layer_nums

        self.output_layer=keras.layers.Dense(7,activation="softmax")

    def call(self,inputs):
        deep_input,wide_input=inputs

        for _ in range(self.deep_num):
            deep_input=self.deep_hidden_layer(deep_input)
        for _ in range(self.wide_num):
            wide_input=self.wide_hidden_layer(wide_input)

        outputs=keras.layers.concatenate([deep_input,wide_input])

        outputs=self.output_layer(outputs)

        return outputs        

In [156]:
wd_model=WD_Model([2,2])

모델을 훈련시는 단계는 간단합니다.

1. compile : 실행환경을 만듭니다
2. fit : 실행환경 하에 학습을 시작합니다.

### Compile

아래 각각의 요소는 다음과 같이 호출해 사용가능합니다.

In [183]:
# gradient를 통한 변화 적용방법 -> optimizer
optimizer=keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)

# gradient를 계산할 손실함수
# 위에서 우리는 label을 정수인코딩을 했습니다(다중클래스 분류)
loss=keras.losses.SparseCategoricalCrossentropy()

# 성능을 확인할 척도
metric_list=[keras.metrics.CategoricalAccuracy()]

wd_model.compile(optimizer=optimizer,loss=loss,metrics=metric_list)

### fit

학습시 호출할 중요한 파라미터 중 하나로 `callbacks`에 대해 알아보죠.

In [158]:
# 차후 텐서보드를 이용하기 위해서 아래의 callback 요소를 꼭 기억해야합니다
keras.callbacks.TensorBoard

# 학습 조기 종료 및 최고 성능 모델 복원
early_stopping=keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,restore_best_weights=True)

# 학습동안의 최대 성능의 모델 저장
model_checkopint=keras.callbacks.ModelCheckpoint

callback_list=[early_stopping]

In [159]:
processed_train=norm_layer(train_feature[num_cols]),cate_layer(train_feature[cate_cols])
processed_test=norm_layer(test_feature[num_cols]),cate_layer(test_feature[cate_cols])

In [160]:
wd_model.fit(
    x=processed_train,
    y=train_label.values,
    batch_size=16,
    epochs=10,
    callbacks=callback_list,
    validation_data=(processed_test,test_label.values))

Epoch 1/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - categorical_accuracy: 0.0991 - loss: 1.6901 - val_categorical_accuracy: 0.0984 - val_loss: 1.5789
Epoch 2/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - categorical_accuracy: 0.0948 - loss: 1.5173 - val_categorical_accuracy: 0.0846 - val_loss: 1.5244
Epoch 3/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - categorical_accuracy: 0.0923 - loss: 1.5776 - val_categorical_accuracy: 0.0839 - val_loss: 2.2229
Epoch 4/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - categorical_accuracy: 0.0914 - loss: 1.6579 - val_categorical_accuracy: 0.0785 - val_loss: 2.2629
Epoch 5/10
5417/5417 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - categorical_accuracy: 0.0947 - loss: 2.7325 - val_categorical_accuracy: 0.1024 - val_loss: 4.3914


이 후 전처리 layer를 합쳐주겠습니다.

In [165]:
inputs=(keras.layers.Input(shape=(9,)),keras.layers.Input(shape=(4,)))
processed_input=(norm_layer(inputs[0]),cate_layer(inputs[1]))
outputs=wd_model(processed_input)

final_wd_model=keras.Model(inputs=inputs,outputs=outputs)